In [1]:
!ls ../data/SRAD2018/

SRAD2018_Test_1     SRAD2018_TRAIN_003	SRAD2018_TRAIN_006  SRAD2018_TRAIN_009
SRAD2018_TRAIN_001  SRAD2018_TRAIN_004	SRAD2018_TRAIN_007  zip
SRAD2018_TRAIN_002  SRAD2018_TRAIN_005	SRAD2018_TRAIN_008


Probably merge datasets or map them into logical one

Use CSV file?

In [2]:
num_datasets = 1

In [3]:
dataset_idx = range(num_datasets)

In [4]:
len(dataset_idx)

1